In [1]:
#Import libraries

# packages for Vietnamese
!pip install underthesea

import requests
import nltk
import math
import re
import regex as re
import pandas as pd
import numpy as np
import statistics as stats
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import json

#You will need to download some packages from NLTK. 

from bs4 import BeautifulSoup 
from nltk.corpus import stopwords

#In most environments, you will need to install NER-D. 

!pip install ner-d
from nerd import ner

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import MiniBatchKMeans
from sklearn.feature_extraction.text import TfidfVectorizer

     |████████████████████████████████| 7.6 MB 4.6 MB/s 
     |████████████████████████████████| 235 kB 48.1 MB/s 
     |████████████████████████████████| 581 kB 41.4 MB/s 
     |████████████████████████████████| 965 kB 48.2 MB/s 
     |████████████████████████████████| 10.3 MB 5.5 MB/s 
     |████████████████████████████████| 2.2 MB 33.6 MB/s 
  Attempting uninstall: thinc
    Found existing installation: thinc 7.4.0
    Uninstalling thinc-7.4.0:
      Successfully uninstalled thinc-7.4.0
  Attempting uninstall: spacy
    Found existing installation: spacy 2.2.4
    Uninstalling spacy-2.2.4:
      Successfully uninstalled spacy-2.2.4


In [2]:
#Initialize data fields for final dataset

dates=[]
titles=[]
content_text=[]
links=[]
keywords=[]
topic_categories=[]

#initializing clustering variables for later topic modeling

cluster_keywords=[]
cluster_number=[]

In [3]:
def get_nav(list):
  page=requests.get('https://batdongsan.vn/')
  soup=BeautifulSoup(page.text,'html.parser')
  container=soup.find_all('li',{'class':'uk-parent'})
  container_a= [cont.find_all('a') for cont  in container]
  ahref = []
  for i in list:
    print(container_a[i])
    ahref += container_a[i][1:]
  # ahref = container_a[5][1:]+container_a[3][1:]+container_a[6][1:]
  links = [link.get('href') for link in ahref]
  return links

In [ ]:
topics = get_nav([3,5,6])
print(topics)
urls=[]
hrefs = []

[<a href="tin-tuc">TIN TỨC <i class="uk-icon-angle-down"></i></a>, <a href="tin-tuc/nhin-ra-the-gioi">Nhìn ra Thế giới</a>, <a href="tin-tuc/chuyen-dong-thi-truong">Chuyển động Thị trường</a>, <a href="tin-tuc/chinh-sach-quan-ly">Chính sách Quản lý</a>, <a href="tin-tuc/thong-tin-quy-hoach">Thông tin Quy hoạch</a>, <a href="tin-tuc/giai-phap-cong-nghe">Giải pháp Công nghệ</a>, <a href="tin-tuc/hoat-dong-doanh-nghiep">Hoạt động Doanh nghiệp</a>, <a href="tin-tuc/thong-tin-su-kien">Thông tin Sự kiện</a>]
[<a href="chuyen-de">CHUYÊN ĐỀ <i class="uk-icon-angle-down"></i></a>, <a href="chuyen-de/bat-dong-san-so">Bất động sản số</a>, <a href="chuyen-de/he-sinh-thai-bat-dong-san">Hệ sinh thái Bất động sản</a>, <a href="chuyen-de/khoi-nghiep-bat-dong-san">Khởi nghiệp Bất động sản</a>, <a href="chuyen-de/van-de-va-giai-phap">Vấn đề và Giải pháp</a>, <a href="chuyen-de/bat-dong-san-thoi-covid">Bất động sản thời Covid</a>]
[<a href="thu-vien">THƯ VIỆN <i class="uk-icon-angle-down"></i></a>, <a hr

In [6]:
def get_hrefs(page,class_name):
  page=requests.get(page)
  soup=BeautifulSoup(page.text,'html.parser')
  container=soup.find_all('div',{'class':class_name})
  container_a=container[0].find_all('a')
  links=[container_a[i].get('href') for i in range(len(container_a))]
  for link in links:
    #print(link)
    if link==None:
      continue
    if 'topic' not in link and 'author' not in link and 'https' in link:
      hrefs.append(link)

def get_topic_len(page,class_name):
  page=requests.get(page)
  soup=BeautifulSoup(page.text,'html.parser')
  container=soup.find_all('ul',{'class':class_name})
  if not len(container):
    return 0
  container_a=container[0].find_all('li')
  return int(container_a[-1].find('a').get('data-ci-pagination-page'))

In [ ]:
url = 'https://batdongsan.vn/'
for topic in topics:
  n_p = get_topic_len(url+topic, 'uk-pagination')
  topic_url = url+topic+'/'
  if n_p == 0:
    urls.append(topic_url)
  else:
    urls.append(topic_url)
    for i in range(1,n_p-1):
      urls.append(topic_url+'p'+str(i+1))

for url in urls:
  print(url)
  get_hrefs(url,'uk-grid uk-grid-collapse uk-grid-width-1-1')
hrefs=set(hrefs)
hrefs

https://batdongsan.vn/tin-tuc/nhin-ra-the-gioi/
https://batdongsan.vn/tin-tuc/nhin-ra-the-gioi/p2
https://batdongsan.vn/tin-tuc/nhin-ra-the-gioi/p3
https://batdongsan.vn/tin-tuc/nhin-ra-the-gioi/p4
https://batdongsan.vn/tin-tuc/chuyen-dong-thi-truong/
https://batdongsan.vn/tin-tuc/chuyen-dong-thi-truong/p2
https://batdongsan.vn/tin-tuc/chuyen-dong-thi-truong/p3
https://batdongsan.vn/tin-tuc/chuyen-dong-thi-truong/p4
https://batdongsan.vn/tin-tuc/chinh-sach-quan-ly/
https://batdongsan.vn/tin-tuc/chinh-sach-quan-ly/p2
https://batdongsan.vn/tin-tuc/chinh-sach-quan-ly/p3
https://batdongsan.vn/tin-tuc/chinh-sach-quan-ly/p4
https://batdongsan.vn/tin-tuc/thong-tin-quy-hoach/
https://batdongsan.vn/tin-tuc/thong-tin-quy-hoach/p2
https://batdongsan.vn/tin-tuc/thong-tin-quy-hoach/p3
https://batdongsan.vn/tin-tuc/thong-tin-quy-hoach/p4


KeyboardInterrupt: ignored

In [ ]:
#Function to extract publication data

def get_date(soup):
  try:
    data=soup.find('div',{'class':'meta'}).get_text()
    dates.append(data.replace('\n', ''))
  except Exception:
    dates.append('')
    pass

#Function to extract product title
def get_title(soup):
  try:
    title=soup.find('title').contents
    titles.append(title[0])
  except Exception:
    titles.append('')
    pass

#Function to extract product's text contents
def get_contents(soup):
  try:
    content = soup.find('div',{'class':'uk-panel'}).get_text()
    lines = (line.strip() for line in content.splitlines())
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    content = '\n'.join(('\n'.join(chunk for chunk in chunks if chunk)).split("\n") [2:-2])
    content_text.append(content)
  except Exception:
    content_text.append('')

In [ ]:
#Iterate through the hrefs extracted from 'browse', extract pertinent content 

iteration=1

#The first few functions rely on the original extracted web content as a parameter.
#These are all basic web-scraping techniques. 
content_text.clear()
dates.clear()
titles.clear()
linklist = [link for link in hrefs if not "chu-dau-tu" in link]

for href in linklist:
  print('Web scraping: iteration',iteration,'/',len(linklist))
  page=requests.get(href)
  soup=BeautifulSoup(page.text,'html.parser')
  links.append(href)
  get_date(soup)
  get_title(soup)
  get_contents(soup)
  iteration+=1

Web scraping: iteration 1 / 428
Web scraping: iteration 2 / 428
Web scraping: iteration 3 / 428
Web scraping: iteration 4 / 428
Web scraping: iteration 5 / 428
Web scraping: iteration 6 / 428
Web scraping: iteration 7 / 428
Web scraping: iteration 8 / 428
Web scraping: iteration 9 / 428
Web scraping: iteration 10 / 428
Web scraping: iteration 11 / 428
Web scraping: iteration 12 / 428
Web scraping: iteration 13 / 428
Web scraping: iteration 14 / 428
Web scraping: iteration 15 / 428
Web scraping: iteration 16 / 428
Web scraping: iteration 17 / 428
Web scraping: iteration 18 / 428
Web scraping: iteration 19 / 428
Web scraping: iteration 20 / 428
Web scraping: iteration 21 / 428
Web scraping: iteration 22 / 428
Web scraping: iteration 23 / 428
Web scraping: iteration 24 / 428
Web scraping: iteration 25 / 428
Web scraping: iteration 26 / 428
Web scraping: iteration 27 / 428
Web scraping: iteration 28 / 428
Web scraping: iteration 29 / 428
Web scraping: iteration 30 / 428
Web scraping: itera

In [ ]:
# save to csv
import pandas as pd
df = pd.DataFrame({"date": dates,"title" : titles, "content": content_text,"hrefs": linklist})
df.dropna(how='any')


,date,title,content,hrefs
0,14:41 30/07/2021,Sử dụng trần nhà màu đen để gia tăng hiệu ứng ...,"Thông thường, phần lớn mọi người hay chọn nhữn...",https://batdongsan.vn/thu-vien/su-dung-tran-nh...
1,10:34 08/09/2021,Ứng dụng đầu tư và tích lũy Infina gọi vốn thà...,Ứng dụng đầu tư và tích lũy Infina mới đây côn...,https://batdongsan.vn/chuyen-de/ung-dung-dau-t...
2,02:51 22/12/2021,Long An được chuyển 232 héc-ta đất trồng lúa đ...,Chính phủ đã cho phép UBND tỉnh Long An chuyển...,https://batdongsan.vn/tin-tuc/long-an-duoc-chu...
3,14:30 23/08/2021,Kinh doanh Bất động sản thời kỳ 4.0,Với xu hướng chuyển đổi số trong thời đại công...,https://batdongsan.vn/tin-tuc/kinh-doanh-bat-d...
4,18:19 07/09/2021,Tòa nhà chọc trời 'mỏng' nhất thế giới,"Vào ngày 1-9 vừa qua, Tập đoàn Phát triển bất ...",https://batdongsan.vn/tin-tuc/toa-nha-choc-tro...
...,...,...,...,...
423,20:04 27/09/2021,Bà Rịa - Vũng Tàu thu hồi dự án 100ha 'ngủ quê...,Dự án đầu tư xây dựng khu công viên hồ Rạch Bà...,https://batdongsan.vn/tin-tuc/ba-ria---vung-ta...
424,17:01 14/07/2021,20 phong cách màu sắc ngoại thất đẹp trên thế ...,Lựa chọn màu sắc có tác động đáng kể đến giá t...,https://batdongsan.vn/thu-vien/20-phong-cach-m...
425,09:28 09/02/2022,"Sau Tân Hoàng Minh, thêm doanh nghiệp bỏ cọc l...",TPHCM - Công ty TNHH Đầu tư Phát triển và Thươ...,https://batdongsan.vn/tin-tuc/sau-tan-hoang-mi...
426,10:07 11/02/2022,Bất động sản Hải Phòng dự báo tăng trưởng m...,Đây là dự báo của Hiệp hội Bất động sản ...,https://batdongsan.vn/tin-tuc/bat-dong-san-hai...


In [ ]:
df.to_csv("drive/Shareddrives/CNPM_mr/bdsvn.csv")

In [ ]:
df_1 = pd.read_csv("drive/Shareddrives/CNPM_mr/bdsvn.csv")


0      14:41 30/07/2021
1      10:34 08/09/2021
2      02:51 22/12/2021
3      14:30 23/08/2021
4      18:19 07/09/2021
             ...       
423    20:04 27/09/2021
424    17:01 14/07/2021
425    09:28 09/02/2022
426    10:07 11/02/2022
427    08:33 08/02/2022
Name: Date, Length: 428, dtype: object

# Lấy thông tin các bất động sản


In [4]:
topics = get_nav([0,1])
print(topics)
urls=[]
hrefs = []

[<a href="ban-nha-dat">MUA BÁN <i class="uk-icon-angle-down"></i></a>, <a href="ban-nha">Bán Nhà</a>, <a href="ban-dat">Bán Đất</a>, <a href="ban-biet-thu">Bán Biệt thự</a>, <a href="ban-bds-thuong-mai">Bán BĐS Thương mại</a>, <a href="ban-can-ho-chung-cu">Bán Căn hộ chung cư</a>, <a href="ban-van-phong">Bán Văn phòng</a>, <a href="ban-bds-cong-nghiep">Bán BĐS Công nghiệp</a>, <a href="ban-bds-nong-nghiep">Bán BĐS Nông nghiệp</a>, <a href="ban-bds-tam-linh">Bán BĐS Tâm linh</a>, <a href="ban-bds-khac">Bán BĐS Khác</a>]
[<a href="cho-thue-nha-dat">CHO THUÊ <i class="uk-icon-angle-down"></i></a>, <a href="cho-thue-nha">Cho thuê Nhà</a>, <a href="cho-thue-dat">Cho thuê Đất</a>, <a href="cho-thue-can-ho-chung-cu">Cho thuê Căn hộ chung cư</a>, <a href="cho-thue-van-phong">Cho thuê Văn phòng</a>, <a href="cho-thue-biet-thu">Cho thuê Biệt thự</a>, <a href="cho-thue-bds-thuong-mai">Cho thuê BĐS Thương mại</a>, <a href="cho-thue-bds-cong-nghiep">Cho thuê BĐS Công nghiệp</a>, <a href="cho-thue-b

In [7]:
url = 'https://batdongsan.vn/'
urls.clear()
lim = 10
from multiprocessing import Pool
from itertools import chain
def get_url(topic):
  n_p = get_topic_len(url+topic, 'uk-pagination')
  _urls = []
  # Just get top 10 page
  if n_p >= lim:
    n_p = lim
  topic_url = url+topic+'/'
  if n_p == 0:
    _urls.append(topic_url)
  else:
    _urls.append(topic_url)
    for i in range(1,n_p-1):
      _urls.append(topic_url+'p'+str(i+1))
  return _urls

p = Pool(10)
urls = p.map(get_url, topics)
p.terminate()
p.join()
urls = list(chain.from_iterable(urls))
print(urls)

['https://batdongsan.vn/ban-nha/', 'https://batdongsan.vn/ban-nha/p2', 'https://batdongsan.vn/ban-nha/p3', 'https://batdongsan.vn/ban-nha/p4', 'https://batdongsan.vn/ban-nha/p5', 'https://batdongsan.vn/ban-nha/p6', 'https://batdongsan.vn/ban-nha/p7', 'https://batdongsan.vn/ban-nha/p8', 'https://batdongsan.vn/ban-nha/p9', 'https://batdongsan.vn/ban-dat/', 'https://batdongsan.vn/ban-dat/p2', 'https://batdongsan.vn/ban-dat/p3', 'https://batdongsan.vn/ban-dat/p4', 'https://batdongsan.vn/ban-dat/p5', 'https://batdongsan.vn/ban-dat/p6', 'https://batdongsan.vn/ban-dat/p7', 'https://batdongsan.vn/ban-dat/p8', 'https://batdongsan.vn/ban-dat/p9', 'https://batdongsan.vn/ban-biet-thu/', 'https://batdongsan.vn/ban-biet-thu/p2', 'https://batdongsan.vn/ban-biet-thu/p3', 'https://batdongsan.vn/ban-biet-thu/p4', 'https://batdongsan.vn/ban-biet-thu/p5', 'https://batdongsan.vn/ban-biet-thu/p6', 'https://batdongsan.vn/ban-biet-thu/p7', 'https://batdongsan.vn/ban-biet-thu/p8', 'https://batdongsan.vn/ban-bi

In [8]:
hrefs.clear()

def extract_href(url):
  try:
    _hrefs = []
    page=requests.get(url)
    soup=BeautifulSoup(page.text,'html.parser')
    container=soup.find_all('div',{'class':'uk-grid uk-grid-small uk-grid-width-1-1'})
    container_a=container[0].find_all('a')
    links=[container_a[i].get('href') for i in range(len(container_a))]
    for link in links:
      # print(link)
      if link==None:
        continue
      if 'topic' not in link and 'author' not in link and 'https' in link:
        _hrefs.append(link)
    return _hrefs
  except:
    return []
p = Pool(10)
hrefs += p.map(extract_href, urls)
p.terminate()
p.join()
hrefs = list(chain.from_iterable(hrefs))
print(len(hrefs))
    

6139


In [9]:
  
import re
hrefs=list(set(hrefs))
hrefs = [href for href in hrefs if not re.match('https://batdongsan.vn/([0-9]+|[a-z]+-[0-9]+|[a-z0-9]+$)',href)]
print(len(hrefs))

2025


In [10]:
price = []
S = []
bedrooms = []
WCs = []
Address = []
Direction= []
news_code = []
dates = []

contents = []
titles = []
links = []

In [ ]:
price.clear()
S.clear()
bedrooms.clear()
WCs.clear()
Address.clear()
Direction.clear()
news_code.clear()
dates.clear()
links.clear()
contents.clear()
titles.clear()
import re

def get_bds_info(soup):
  try:
    _price = soup.find('strong',{'class' : 'price'}).get_text()
    price.append(_price)
  except:
    price.append('')
  info = soup.find_all('div',{'class' : 'param'})
  info_li = []
  for div in info:
    info_li += div.find_all('li')

  S.append('')
  bedrooms.append('')
  WCs.append('')
  Address.append('')
  news_code.append('')
  dates.append('')
  Direction.append('')
  for in4 in info_li:
    try:
      title = in4.find('strong').get_text()
      cont = in4.get_text().replace(title,"")
      if "Diện tích" in title:
        S[-1] = re.findall('[0-9]+', cont)[0]
      elif "Phòng ngủ" in title:
        bedrooms[-1] = re.findall('[0-9]+', cont)[0]
      elif "Phòng WC" in title:
        WCs[-1] = re.findall('[0-9]+', cont)[0]
      elif "Địa chỉ" in title:
        Address[-1] = cont
      elif "Mã tin" in title:
        news_code[-1] = cont
      elif "Ngày đăng" in title:
        dates[-1] = cont
      elif "Hướng nhà" in title:
        Direction[-1] = cont
    except:
      continue

#Function to extract product title
def get_bds_title(soup):
  try:
    title=soup.find('h1',{'class':'uk-panel-title'}).get_text()
    titles.append(title.strip())
  except Exception:
    titles.append('')
    pass

#Function to extract product's text contents
def get_bds_contents(soup):
  try:
    content = soup.find('div',{'class':'content'}).get_text().strip()
    content = re.sub("[0-9]+|- ","",content)
    content = re.sub(" +"," ",content)
    contents.append(content)
  except Exception:
    contents.append('')

hrefs = list(hrefs)
iteration = 1
limit = 10
for href in hrefs:
  print('Web scraping: iteration',iteration,'/',limit)
  print(href)
  links.append(href)
  page=requests.get(href)
  soup=BeautifulSoup(page.text,'html.parser')
  get_bds_info(soup)
  get_bds_title(soup)
  get_bds_contents(soup)
  iteration += 1
  if iteration == limit:
    break


  

Web scraping: iteration 1 / 10
https://batdongsan.vn/ban-dat-thanh-da-gia-cuc-re-r77954
Web scraping: iteration 2 / 10
https://batdongsan.vn/ban-shophouse-can-goc-tai-du-an-dolphin-plaza-28-tran-binh-my-dinh-gan-cau-giay-hn-7t-thang-may-r70127
Web scraping: iteration 3 / 10
https://batdongsan.vn/can-ban-gap-3-sap-ca-cho-hoa-hung-r32577
Web scraping: iteration 4 / 10
https://batdongsan.vn/dat-co-so-28x36-m-tp-thu-duc-tphcm-r46715
Web scraping: iteration 5 / 10
https://batdongsan.vn/cho-thue-dat-1030-tien-lam-kho-xuong-san-xuat-24t-r29018
Web scraping: iteration 6 / 10
https://batdongsan.vn/cho-thue-van-phong-280m2-pho-xa-dan-dong-da-ha-noi-gia-chi-48trth-lh-0972709960-r77298
Web scraping: iteration 7 / 10
https://batdongsan.vn/can-ban-gap-can-ho-cao-cap-masteri-thao-dien-quan-2-hcm-r78053
Web scraping: iteration 8 / 10
https://batdongsan.vn/cho-thue-van-phong-vi-tri-dep-kd-tot-gia-re-15m2-30m2-gia-25tr-5trthang-r77660
Web scraping: iteration 9 / 10
https://batdongsan.vn/ngoc-ha-ba-dinh-

In [ ]:
import pandas as pd
df = pd.DataFrame({ "price":price,"S(m2)":S,"bedrooms":bedrooms,"WCs":WCs,"Address":Address,"Direction":Direction,"news_code":news_code,"dates":dates,"contents":contents,"titles":titles,"link":links})
df.columns

Index(['price', 'S(m2)', 'bedrooms', 'WCs', 'Address', 'Direction',
       'news_code', 'dates', 'contents', 'titles', 'link'],
      dtype='object')

In [ ]:
excep = "BATDONGSAN.VN"
df = df[df["contents"].str.contains(excep)==False]
df.reset_index(inplace=True)
df

NameError: ignored

# Multiprocessing Crawler

In [11]:
print(len(hrefs))

2025


In [13]:

from multiprocessing import Pool,Manager
from random import sample

def scrape_url(href):
  print(href)
  page=requests.get(href)
  soup=BeautifulSoup(page.text,'html.parser')
 
  try:
    _price = soup.find('strong',{'class' : 'price'}).get_text()
    price = _price
  except:
    price = ''

  info = soup.find_all('div',{'class' : 'param'})
  info_li = []
  for div in info:
    info_li += div.find_all('li')

  S = ''
  bedrooms = ''
  WCs= ''
  Address = ''
  news_code  = ''
  dates = ''
  Direction = ''
  for in4 in info_li:
    try:
      title = in4.find('strong').get_text()
      cont = in4.get_text().replace(title,"")
      if "Diện tích" in title:
        S = re.findall('[0-9]+', cont)[0]
      elif "Phòng ngủ" in title:
        bedrooms= re.findall('[0-9]+', cont)[0]
      elif "Phòng WC" in title:
        WCs = re.findall('[0-9]+', cont)[0]
      elif "Địa chỉ" in title:
        Address = cont
      elif "Mã tin" in title:
        news_code = cont
      elif "Ngày đăng" in title:
        dates = cont
      elif "Hướng nhà" in title:
        Direction = cont
    except:
      continue



  # get_bds_title(soup)
  try:
    title=soup.find('h1',{'class':'uk-panel-title'}).get_text()
    titles = title.strip()
  except Exception:
    titles = ''
    pass


  # get_bds_contents(soup)
  try:
    content = soup.find('div',{'class':'content'}).get_text().strip()
    contents = content
  except Exception:
    contents = ''

  return (price,S,bedrooms,WCs,Address,Direction,news_code,dates,contents,titles,href)

p = Pool(20)
bds_list = p.map(scrape_url, hrefs)
p.terminate()
p.join()



https://batdongsan.vn/ban-nha-quan-3-chinh-chu-cu-xa-do-thanh-gia-re-nha-dep-r79326
https://batdongsan.vn/ban-nha-mat-tien-le-van-sy-r78597
https://batdongsan.vn/cho-thue-181-duong-ba-thang-hai-quan-10-r77718
https://batdongsan.vn/cho-thue-nha-rieng-dt-65m2-x-4t-tai-dang-van-ngu-gia-chi-tu-20trthang-r77546
https://batdongsan.vn/can-von-ban-can-shop-gan-truc-duong-36m-gia-76-ty-r76647
https://batdongsan.vn/cho-thue-mbkd-van-phong-dao-tao-gym-495m2-hoa-binh-green-505-minh-khai-gia-chi-140ngm2thg-r77508
https://batdongsan.vn/dat-nen-shr-xa-huu-thanh-duc-hoa-hem-xe-hoi-ngay-kcn-idico-800ha-r78980
https://batdongsan.vn/cho-thue-ch-tai-the-sun-avenue-1pn-nha-moi-huong-dn-tien-ich-view-landmark-lh-ngay-0967927823-r77730
https://batdongsan.vn/chinh-chu-ban-c23-0x-dt-400m2-gia-40-trm2-va-c5-1x-mat-cong-vien-gia-46-trm2-r77931
https://batdongsan.vn/ban-nha-q-12-biet-thu-ven-song-du-thuyen-cap-nha-co-ho-boi-hang-hiem-ms-672-r78425
https://batdongsan.vn/chi-can-30-trieu-co-ngay-biet-thu-duong-noi-

Process ForkPoolWorker-73:
Process ForkPoolWorker-66:
Process ForkPoolWorker-61:
Process ForkPoolWorker-65:
Process ForkPoolWorker-72:
Process ForkPoolWorker-74:
Process ForkPoolWorker-70:
Process ForkPoolWorker-63:
Process ForkPoolWorker-69:
Process ForkPoolWorker-79:
Process ForkPoolWorker-64:
Process ForkPoolWorker-62:
Process ForkPoolWorker-67:
Process ForkPoolWorker-78:
Process ForkPoolWorker-77:
Process ForkPoolWorker-75:
Process ForkPoolWorker-80:
Traceback (most recent call last):
Process ForkPoolWorker-68:
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-76:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process ForkPoolWorker-71:
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lo

KeyboardInterrupt: ignored

  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)

During handling of the above exception, another exception occurred:

  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/usr/lib/python3.7/multiprocessing/process.py", line 297, in _bootstrap
    self.run()
  File "/usr/lib/python3.7/multiprocessing/pool.py", line 121, in worker
    result = (True, func(*args, **kwds))
TypeError: getresponse() got an unexpected keyword argument 'buffering'
  File "/usr/lib/python3.7/multiprocessing/process.py", line 99, in run
    self._target(*self._args, **self._kwargs)
TypeError: getresponse() got an unexpected keyword argument 'buffering'
TypeError: getresponse() got an unexpected keyword argument 'buffering'
TypeError: getrespons

In [ ]:
print("\t\t\t\tb\t\t\t\n".replace("\t","").replace("\n",""))

b


In [ ]:
import pandas as pd

df = pd.DataFrame(bds_list)
df.columns = ["price","S(m2)","bedrooms","WCs","Address","Direction","news_code","dates","contents","titles","link"]


In [ ]:
df.price = df.price.str.replace("\n","")
df.price = df.price.str.replace("\t","")
df["price"]

0              1.23 tỷ
1              13.5 tỷ
2       12 triệu/tháng
3       10 triệu/tháng
4           Thỏa thuận
             ...      
1979     4 triệu/tháng
1980    70 nghìn/tháng
1981            7.5 tỷ
1982           9.55 tỷ
1983            2.5 tỷ
Name: price, Length: 1984, dtype: object

In [ ]:
df.to_csv("drive/Shareddrives/CNPM_mr/bdsvn_bdsin4.csv")

In [14]:
import pandas as pd
df = pd.read_csv("drive/Shareddrives/CNPM_mr/bdsvn_bdsin4.csv")

In [15]:
df

,Unnamed: 0,price,S(m2),bedrooms,WCs,Address,Direction,news_code,dates,contents,titles,link
0,0,1.23 tỷ,66.0,2.0,2.0,Xã Cự Khê Huyện Thanh Oai Hà Nội,Nam,78157.0,15:22 09/04/2022,Căn hộ tại tầng 5 cụm 5 tòa mặt đường có diện...,Chính chủ bán căn hộ 2PN toà HH02C KĐT Thanh H...,https://batdongsan.vn/chinh-chu-ban-can-ho-2pn...
1,1,13.5 tỷ,60.0,4.0,NaN,347 hoàng quốc việt,NaN,78474.0,08:57 10/04/2022,Bán nhà mặt ngõ 347 Hoàng Quốc Việt – gần chợ ...,"Bán nhà PL Hoàng Quốc Việt – Kinh doanh đỉnh, ...",https://batdongsan.vn/ban-nha-pl-hoang-quoc-vi...
2,2,12 triệu/tháng,140.0,4.0,3.0,"Đường Lạc Long Quân, Tân Bình, Hồ Chí Minh",NaN,77870.0,18:42 09/03/2022,cho thuê nhà rất đẹp.ngang trung tâm chợ tân b...,CHO THUÊ NHÀ RẤT ĐẸP NGANG CHỢ TÂN BÌNH 12 TRI...,https://batdongsan.vn/cho-thue-nha-rat-dep-nga...
3,3,10 triệu/tháng,100.0,3.0,1.0,"Ngõ 27, Đường Cát Linh, Phường Cát Linh, Đốn...",NaN,77909.0,19:26 09/03/2022,Căn hộ đầu hồi 100m2 tầng 3 - tại ngõ 27 Cát L...,Chính chủ cho thuê căn hộ 100m2: 3PN 1WC full ...,https://batdongsan.vn/chinh-chu-cho-thue-can-h...
4,4,Thỏa thuận,150.0,NaN,NaN,"Dự án FLC Quảng Bình, Xã Hải Ninh, Quảng Ninh...",NaN,74956.0,10:27 08/03/2022,Là khu nghỉ dưỡng sinh thái mặt biển bao quanh...,Trời ơi tin được không FLC Quảng Bình mở bán đ...,https://batdongsan.vn/troi-oi-tin-duoc-khong-f...
...,...,...,...,...,...,...,...,...,...,...,...,...
1979,1979,4 triệu/tháng,100.0,NaN,NaN,"Đường Phan Khôi, Phường Hòa Xuân, Quận Cẩm Lệ...",NaN,34812.0,16:20 21/02/2022,Cho thuê nhà cấp 4 kiên cố Mới 100% đường Phan...,Cho thuê nhà cấp 4 đường Phan Khôi mới xây,https://batdongsan.vn/cho-thue-nha-cap-4-duong...
1980,1980,70 nghìn/tháng,5000.0,NaN,NaN,"Xã Hà Hồi, Thường Tín, Hà Nội",NaN,75693.0,17:23 08/03/2022,Cho thuê kho xưởng DT 1000m2 - 5000m2 KCN Hà B...,Cho thuê kho xưởng DT 1000m2 - 5000m2 KCN Hà B...,https://batdongsan.vn/cho-thue-kho-xuong-dt-10...
1981,1981,7.5 tỷ,472.0,4.0,NaN,NaN,NaN,77979.0,09:40 18/12/2021,"-Chính chủ cần bán gấp nhà phố Đặng Thùy Trâm,...",BÁN NHÀ ĐẶNG THÙY TRÂM - PHẠM TUẤN TÀI,https://batdongsan.vn/ban-nha-dang-thuy-tram-p...
1982,1982,9.55 tỷ,64.0,3.0,3.0,"Đặng Thùy Trâm, phường 13 Bình Thạnh",NaN,78309.0,16:05 09/04/2022,+Nhà 4 x 16 vuông vức đang cho thuê trước dịch...,Bình Thạnh Bán Đất Tặng Nhà- Khu Phân Lô – Đườ...,https://batdongsan.vn/binh-thanh-ban-dat-tang-...
